#WHAT DOES MACHINE LEARNING SAY ABOUT BITCOIN PRICE ? 

The fundemental rule of economics says that demand for and price eventually comes at a balance. I think this simple but important aspect of trading is ignored severely by the traders. 

I built this project on this rule. The price of bitcoin is related to the amount of demand in market as any tradable asset. This reality leads me to measure the trend of demand in the market for bitcoin and expect the price go the same way.

The predictions are not solely based on demand but several technical indicators are also taken into consideration as features in the model.
 


##Install and Import Dependencies

In [ ]:
pip install python-binance pandas mplfinance vaderSentiment snscrape pytrends

In [2]:
import pandas as pd
from pytrends.request import TrendReq
import numpy as np
import  snscrape.modules.twitter as sntwitter
from binance import ThreadedDepthCacheManager, ThreadedWebsocketManager, Client      
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer                       
from datetime import datetime,timedelta                 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,recall_score,roc_auc_score, mean_absolute_error,mean_squared_error
import lightgbm as lgb
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import  Sequential
from tensorflow.keras.optimizers import  Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import  LSTM, Dense, Dropout, Input,Flatten, InputLayer

###Preparation of different pieces of data for the project

In this project, I plan to collect data from different sources and consequently create insights by utilizing machine learning models. 

**List of the requirements:**

1.   Google Search Scores from Google Trends
2.   Tweets about bitcoin from Twitter
3.   Bitcoin historical price data from Binance API


In [3]:
secret_key='UaOib04hOwZoFOg11ZHM0oA4jdYNSXudxum5WLVACNO7tKY1MzJRTjgoGr4PW0i0'                # replace it with your own secret key
API_key='EEFDWcSSEMGxCzNDyyFgNuSXToc4UkudnanmjlK1HFU32voQwIVYWY1LuBphui73'                   # replace it with your own API key

start_date =  datetime.now() - timedelta(days = 1500)                                        # the start day is softcoded to be always 1500 days before the current date
start_date = start_date.date()                                                               # extract the date from datetime format
end_date = datetime.now().date()                                                             # extract the date from datetime format

### TWITTER SCRAPING

In [ ]:
# query is the exact string when you perform an advanced search on twitter.com. Topic,date,language,location... all can be selected.
query = f'bitcoin min_faves:998 lang:en until:{end_date} since:{start_date} -filter:links'     
tweets = []                                 # empty list to gather all data about tweets
limit = 50000                               # maximum number of tweets to gather
for tweet in sntwitter.TwitterSearchScraper(query).get_items():               # loop through the items
    if len(tweets) == limit:                # stops gathering data if the limit is reached
        break
    elif tweet.likeCount > 1000:            # gathers only the tweets with more than 1000 likes - It is assumed that tweets with high  number of likes represent a stronger sentiment
        tweets.append([tweet.date,tweet.content, tweet.likeCount])            # add data of each tweet to the list

sentiment = pd.DataFrame(tweets, columns=['Date''Tweet','Likes'])      # create a data frame out of the list.

In [ ]:
# df.to_csv('tweets.csv')                    # save the data frame     
sentiment = pd.read_csv('tweets.csv')        # read the saved data
sentiment['Date'] = sentiment['Date'].astype('datetime64').apply(lambda x: x.strftime(format='%Y-%m-%d'))

In [ ]:
#calculate the negative, positive, neutral and compound scores, plus verbal evaluation
def sentiment_vader(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()                         # this is a pretrained ready-to-use sentiment analysis model.

    sentiment_dict = sid_obj.polarity_scores(sentence)             # make polarity calculation
    negative = sentiment_dict['neg']                               # assign the negative value to a variable
    neutral = sentiment_dict['neu']                                # assign the neutral value to a variable
    positive = sentiment_dict['pos']                               # assign the positive value to a variable
    compound = sentiment_dict['compound']                          # assign the overall value to a variable

    if sentiment_dict['compound'] >= 0.05 :                        # if the overall value is higher than 0.05 it is considered as positive
        overall_sentiment = "Positive"

    elif sentiment_dict['compound'] <= - 0.05 :                    # if the overall value is lower than 0.05 it is considered as negative
        overall_sentiment = "Negative"

    else :
        overall_sentiment = "Neutral"                              # else it is neutral
  
    return negative, neutral, positive, compound, overall_sentiment   # the function returns 5 different values

Now it is time to extract numbers out of tweets. I use apply function to get sentiment insights from each tweet. Here if the overall sentiment score is labeled with respect to its position on the positive-negative scale.

In [ ]:
sentiment['score'] = sentiment.Tweet.apply(lambda x:sentiment_vader(x)[3])           # Add a new column named 'score' to the data frame

In [ ]:
def conv(x):                                   # create a function labels the scores 
  if x>0:
    x= 'positive'                              # 'positive' if the value is bigger than 0
  elif x<0:
    x = 'negative'                            # 'negative' if the value is smaller than 0
  else:
    x= 'neutral'                              # 'neutral' if the value is  0
  return x    

In [ ]:
sentiment1 = sentiment.copy()
sentiment['label'] = sentiment.score.apply(lambda x:conv(x))     # use the function created above on a lambda function

In [ ]:
sentiment1['Date'] = sentiment1['Date'].astype('datetime64')    # convert the series of object into datetime

In [ ]:
sentiment1 = sentiment1.groupby('Date').sum()                   # group the data by Date and sum the data that belong to the same date.
sentiment1 = sentiment1[['score']]                              # choose only the score column
date_range = pd.date_range(start_date, end_date)                # some dates are missing because there was not any tweet with more than 1000 likes everyday between start date and end date.
# so I create a date range data within the specified period to use it as index
sentiment2 = sentiment1.copy().reindex(date_range,fill_value = 0)      # the date range is assigned as index for the sentiment data frame, the missing dates of the sentiment data is filled with 0
sentiment2.head()                                                


,score
2018-08-16,0.0000
2018-08-17,0.0000
2018-08-18,0.0000
2018-08-19,0.0000
2018-08-20,1.2638


In [ ]:
sentiment2.to_csv('sentiment2.csv')

##GOOGLE SEARCH TRENDS

In this part of the project, I gather google search ranks for each number for a period of 1500 days. I had to split the total time into chunks of 5 months because google trends API offers daily data only if the request covers less than 6 months. Otherwise the data is given as weekly,monthly or yearly. 

First I gathered data for each 5 months and then concatenated all into a single data frame. 

In [ ]:
pytrends = TrendReq(tz=360)                        
# Get Google Keyword Suggestions
kw_list = ['Bitcoin']                                        # list the desired keywords  
# calculate the number of days between the start date and end date, find the number of 5 month periods and round it up. This number will be used in a loop
cycles =round((end_date - start_date).days / 150) + 1        
period_start = start_date                              # starting date of the time block of 5 month
trends = {}                                            # create an empty dictionary to store data in each loop   
for i in range(cycles):                                # loop through the number of cycles
  period_start = start_date + timedelta(days=150*i)    # start of the period equals to the start date and 150 days. the number of extra days is multiplied by 150 each loop
  period_end = period_start + timedelta(days=150)      #  end of the period equals to the start of the period + 150 days
  name = f'trend{i}'                                   # assign a name for each period
  # build_payload function defines the parameters. keyword list equals to kw_list, timeframe is given the period start and period end for each loop. 
  pytrends.build_payload(kw_list=kw_list, cat=0, timeframe='{} {}'.format(period_start.strftime(format = '%Y-%m-%d'),period_end.strftime(format = '%Y-%m-%d')))
   # after the parameters are defined, we call the funciton we want. interest_over_time brings historical data
  trends[name] = pytrends.interest_over_time()          # collected data is stored in 'trends' dictionary

trend_data = pd.concat([trends[x] for x in trends.keys()])     # each list collected in the loop function is concatenated and stored

In [ ]:
trend_data = pd.read_csv('googletrendsdata.csv',index_col='date',parse_dates=True)


In [ ]:
trend_data = trend_data[['Bitcoin']]                     # Only one column is selected
trend_data['yesterday'] = trend_data.shift(1).Bitcoin    # the trend rating of yesterday is added as another column 
trend_data['change'] = trend_data.Bitcoin - trend_data.yesterday      # the change in ratings in a day is calculated

In [ ]:
trend_data = trend_data[~trend_data.index.duplicated()]    # duplicate rows are removed

In [ ]:
date_range = pd.date_range(start_date, end_date)             # created a daily date range to use as index later
trend_data1 = trend_data.copy().reindex(date_range,method = 'nearest')        # merge the trend data frame with the date range. This allows me to have data for each day
                  

,Bitcoin,yesterday,change
2018-08-16,48,56.0,-8.0
2018-08-17,45,48.0,-3.0
2018-08-18,41,45.0,-4.0
2018-08-19,39,41.0,-2.0
2018-08-20,42,39.0,3.0
...,...,...,...
2022-09-20,21,22.0,-1.0
2022-09-21,21,22.0,-1.0
2022-09-22,21,22.0,-1.0
2022-09-23,21,22.0,-1.0


### BINANCE API

I create a client object and pass in my keys to access the data for cryptocurrency pairs. 
I chose to collect daily data since I think it is the most optimal time interval for both technical analysis and sentiment analysis.

In [ ]:
client = Client(API_key,secret_key)                 # client instance is instantiated with personal keys.
tickers = client.get_all_tickers()                  # get the list of trade pairs 
pairs=pd.DataFrame(tickers)                         # create a data frame out of the pairs

In [ ]:
historical_data = client.get_historical_klines('BTCUSDT',Client.KLINE_INTERVAL_1DAY,str(start_date))         # chose BTC-USDT pair and defined the data interval as '1 DAY' starting from my start date.
hist_df = pd.DataFrame(historical_data,columns = ['OpenTime','Open','High','Low','Close','Volume','CloseTime','AssetVolume','NumberofTraders','TakerbaseVol','TakerquoteVol','dro']) # assing column names
del hist_df['dro']                                                                                            # drop an unnecessary column
hist_df['OpenTime']  = pd.to_datetime(hist_df['OpenTime']/1000,unit='s')                                      # convert the unix time format into UTC time format
hist_df['CloseTime']  = pd.to_datetime(hist_df['CloseTime']/1000,unit='s')                                    # convert the unix time format into UTC time format
hist_df[hist_df.select_dtypes(include=['object','int']).columns]=hist_df.select_dtypes(include=['object','int']).astype('float')          #reformatted object and integer columns as float data type
hist_df = hist_df.set_index('OpenTime')                                                                      #  the 'Opentime' column is set as index

In [ ]:
hist_df = hist_df.iloc[:,:5]                                                            # only the first 5 column is filtered
hist_df.head()

,Open,High,Low,Close,Volume
OpenTime,,,,,
2018-08-16,6265.27,6480.00,6205.60,6311.75,48515.254618
2018-08-17,6316.00,6585.00,6285.40,6584.49,57851.610803
2018-08-18,6579.04,6620.00,6288.00,6387.96,53742.322172
2018-08-19,6387.96,6541.00,6300.00,6477.53,45190.847994
2018-08-20,6477.53,6530.00,6220.00,6254.84,49435.555260
...,...,...,...,...,...
2022-09-20,19537.02,19634.62,18711.87,18875.00,324098.328600
2022-09-21,18874.31,19956.00,18125.98,18461.36,385034.100210
2022-09-22,18461.36,19550.17,18356.39,19401.63,379321.721110


##MERGE DATA FRAMES 

In [ ]:
hist_df['tomorrow']=hist_df.Close.shift(-1)                  # the Close column data is shifted one period further
hist_df.dropna(inplace=True)    
hist_df['target'] =  hist_df.tomorrow - hist_df.Close                   # the difference in price between the current day and before that day
hist_df['target'] = hist_df['target'].apply(lambda x : 1 if x > 0 else 0)     # if the price increased since the day before the target column is labeled as 1, if price decreased it is labeled as 0
hist_df                            # drop the null values created by the shift

,Open,High,Low,Close,Volume,tomorrow,target
OpenTime,,,,,,,
2018-08-16,6265.27,6480.00,6205.60,6311.75,48515.254618,6584.49,1
2018-08-17,6316.00,6585.00,6285.40,6584.49,57851.610803,6387.96,0
2018-08-18,6579.04,6620.00,6288.00,6387.96,53742.322172,6477.53,1
2018-08-19,6387.96,6541.00,6300.00,6477.53,45190.847994,6254.84,0
2018-08-20,6477.53,6530.00,6220.00,6254.84,49435.555260,6480.00,1
...,...,...,...,...,...,...,...
2022-09-19,19417.45,19686.20,18232.56,19537.02,380512.403060,18875.00,0
2022-09-20,19537.02,19634.62,18711.87,18875.00,324098.328600,18461.36,0
2022-09-21,18874.31,19956.00,18125.98,18461.36,385034.100210,19401.63,1


In [ ]:
hist_df['50MA'] = hist_df.rolling(50,closed = 'left', min_periods=1).mean().Close         # Take the mean of previous 50 time peroids
hist_df['100MA'] = hist_df.rolling(100,closed = 'left', min_periods=1).mean().Close       # Take the mean of previous 100 time peroids  
hist_df['20MA']= hist_df.rolling(20,closed = 'left', min_periods=1).mean().Close          # Take the mean of previous 20 time peroids 
hist_df['check50MA']=(hist_df.Close - hist_df['50MA']).apply(lambda x : 1 if x > 0 else 0)         # add a column that shows 1 if the price is above 50 moving average, otherwise 0
hist_df['check20MA']=(hist_df.Close - hist_df['20MA']).apply(lambda x : 1 if x > 0 else 0)         # add a column that shows 1 if the price is above 20 moving average, otherwise 0
hist_df['check100MA']=(hist_df.Close - hist_df['100MA']).apply(lambda x : 1 if x > 0 else 0)       # add a column that shows 1 if the price is above 100 moving average, otherwise 0

In [ ]:
merged_data = sentiment2.merge(trend_data1,right_index=True,left_index = True)      # merge the sentiment and trend data under the same index.
merged_data1 = merged_data.copy()                                                   # create copy to work on the data easier
merged_data1.head()

,score,Bitcoin,yesterday,change
2018-08-16,0.0000,48,56.0,-8.0
2018-08-17,0.0000,45,48.0,-3.0
2018-08-18,0.0000,41,45.0,-4.0
2018-08-19,0.0000,39,41.0,-2.0
2018-08-20,1.2638,42,39.0,3.0


In [ ]:
hist_df1=hist_df.copy()
hist_df1 = hist_df1.merge(merged_data1,right_index=True,left_index=True)            # merge the bitcoin price data and the other two data frames
hist_df1['trend20MA'] = hist_df1.Bitcoin.rolling(20,closed = 'left', min_periods=1).mean()     # create 20 period moving averages for trends
hist_df1['trend50MA']= hist_df1.Bitcoin.rolling(50,closed = 'left', min_periods=1).mean()      # create 50 period moving averages for trends
hist_df1['trend100MA']=hist_df1.Bitcoin.rolling(100,closed = 'left', min_periods=1).mean()     # create 100 period moving averages for trends
hist_df1=hist_df1.dropna()                                                           # drop the null values

In [ ]:
hist_df2 = hist_df1.copy()
hist_df1 = hist_df1.drop(columns=['tomorrow'])              # drop the tomorrow column to avoid data leakage in predictions. because it includes data from future.

In [ ]:
hist_df1['score_50MA'] = hist_df1.score.rolling(50,closed='left',min_periods=1).mean()      # create 50 period moving averages for sentiment score
hist_df1['score_100MA'] = hist_df1.score.rolling(100,closed='left',min_periods=1).mean()    # create 50 period moving averages for sentiment score

In [ ]:
# hist_df1.to_csv('all_data.csv')